In [2]:
from geopandas import GeoDataFrame
import pandas as pd

## Juntanto as bases referentes aos setores censitários

In [4]:
df_setores = GeoDataFrame.from_file("setores")

In [9]:
df = pd.read_csv("censo_curitiba.csv",index_col=0)

In [39]:
df_setores.CD_GEOCODI = df_setores.CD_GEOCODI.astype("float64") 

Join entre as bases com base no código do setor para agregar o nome dos bairros aos dados censitários

In [44]:
df_completo = df.set_index("Cod_setor").join(df_setores[["CD_GEOCODI","NM_BAIRRO"]].set_index("CD_GEOCODI"), how="inner")

Garantindo que o nome dos bairros estejam iguais aos nomes da Coleta do OpenStreetMap

In [87]:
bairros = df_completo.NM_BAIRRO.unique()

In [50]:
b = ["ÁGUA VERDE", "JARDIM SOCIAL","XAXIM","JARDIM DAS AMÉRICAS","GUAÍRA","BAIRRO ALTO","SÃO FRANCISCO",
 "ALTO DA GLÓRIA","ATUBA","CABRAL","LINDÓIA","CAMPO COMPRIDO","CENTRO CÍVICO","JUVEVÊ","HUGO LANGE",
 "CAPÃO DA IMBUIA","SEMINÁRIO","GUABIROTUBA","PRADO VELHO","PAROLIN","PORTÃO","VILA IZABEL","CAMPINA DO SIQUEIRA",
 "BOA VISTA","AHÚ","CRISTO REI","REBOUÇAS","ORLEANS","PILARZINHO","MOSSUNGUÊ","ALTO DA RUA XV","BAIRRO JARDIM BOTÂNICO",
 "BUTIATUVINHA","BARREIRINHA","TINGUI","BOQUEIRÃO","RIVIERA","SÍTIO CERCADO","TARUMÃ","SÃO LOURENÇO","BACACHERI",
 "BIGORRILHO","BOM RETIRO","CAPÃO RASO","HAUER","FANNY","NOVO MUNDO","FAZENDINHA","SANTA QUITÉRIA","CENTRO",
 "ABRANCHES","BATEL","SÃO MIGUEL","AUGUSTA","CAXIMBA","ALTO BOQUEIRÃO","PINHEIRINHO","TATUQUARA","UMBARÁ",
 "CAMPO DE SANTANA","GANCHINHO","CAJURU","UBERABA","TABOÃO","CACHOEIRA","SANTA CÂNDIDA","CIDADE INDUSTRIAL DE CURITIBA",
 "LAMENHA PEQUENA","MERCÊS","SANTO INÁCIO","VISTA ALEGRE","SÃO BRAZ","SÃO JOÃO","CASCATINHA","SANTA FELICIDADE"]

In [88]:
a = [b.upper() for b in bairros]

In [89]:
list(set(a) - set(b))

[]

In [90]:
list(set(b) - set(a))

[]

Renomeando os bairros que não estavam iguais aos do OSM

In [86]:
df_completo = df_completo.replace(['Botiatuvinha', 'Tinguí', 'Jardim Botânico', 'Ahu', 'Juveve', 'Mossungue'],
                   ['Butiatuvinha','Tingui','Bairro Jardim Botânico','Ahú','Juvevê','Mossunguê'])

In [92]:
df_completo.to_csv("censo_curitiba_2010.csv", index=False)

## Enriquecendo os bairros com as funcionalidades

In [100]:
from shapely import wkt

In [101]:
df_bairros = pd.read_csv("bairros.csv").rename(columns={"geom":"geometry"})
df_bairros["geometry"]=df_bairros["geometry"].apply(wkt.loads)
df_bairros = GeoDataFrame(df_bairros)

In [248]:
df_bairros

,name,geometry,key,value
0,Bom Retiro,"POLYGON ((-49.28462 -25.41531, -49.28039 -25.4...",boundary,administrative
1,Cajuru,"POLYGON ((-49.23386 -25.44371, -49.23368 -25.4...",boundary,administrative
2,Bairro Jardim Botânico,"POLYGON ((-49.26044 -25.43528, -49.26037 -25.4...",boundary,administrative
3,Boqueirão,"POLYGON ((-49.26199 -25.49131, -49.25913 -25.4...",boundary,administrative
4,Orleans,"POLYGON ((-49.38882 -25.42790, -49.38517 -25.4...",boundary,administrative
...,...,...,...,...
70,Tingui,"POLYGON ((-49.23239 -25.38468, -49.23187 -25.3...",boundary,administrative
71,Atuba,"POLYGON ((-49.21943 -25.38014, -49.21915 -25.3...",boundary,administrative
72,Xaxim,"POLYGON ((-49.28483 -25.50739, -49.28485 -25.5...",boundary,administrative
73,Capão Raso,"POLYGON ((-49.31379 -25.51654, -49.31336 -25.5...",boundary,administrative


In [630]:
def clean_category(function,df_category):
    df_category["geometry"]=df_category["geometry"].apply(wkt.loads)
    df_category = GeoDataFrame(df_category.reset_index(drop=True))
    
    df_category = df_category.reset_index(drop=True)
    
    df_polygon = df_category[df_category.geometry.geom_type == 'Polygon']
    df_point = df_category[df_category.geometry.geom_type != 'Polygon']
    print(len(df_polygon))
    print(len(df_point))
    #remove polygons duplicados
    if (len(df_polygon) != 0):
        df_polygon_res = geopandas.sjoin(df_polygon,df_polygon, how="inner",op="intersects").reset_index()
        index = df_polygon_res[df_polygon_res["index"] < df_polygon_res["index_right"]].index_right.unique()
        df_polygon = df_polygon.drop(index)

        #remove points duplicados
        df_point_res = geopandas.sjoin(df_point,df_polygon, how="inner",op="intersects")
        df_category = df_point.drop(df_point_res.index).append(df_polygon)
    else:
        df_category = df_point
        
    df_category["function"] = [function]*len(df_category)
    
    return df_category
    

In [404]:
import glob

df_roi = GeoDataFrame(columns=["name","geometry","key","value","function"])

for i, file in enumerate(glob.glob('roi_functional/*.csv')):
    function = file[15:-4]
    if function[-1] == "2":
        function = function[:-1]
        
    with open(file) as category_file:
        df_roi =df_roi.append(clean_category(function,pd.read_csv(file,index_col=0).rename(columns={"geom":"geometry"})))
        
        

54
313
68
1197
352
49
414
110
1158
127


In [402]:
df_roi = geopandas.sjoin(df_roi,df_bairros, how="inner",op="intersects")

In [403]:
df_roi = df_roi.rename(columns={"name_left":"LOCAL","function":"FUNCIONALIDADE","name_right":"BAIRRO"}).reset_index(drop=True)

In [264]:
df_roi[["LOCAL","FUNCIONALIDADE","BAIRRO"]].to_csv("roi.csv",index=False)

In [267]:
df_roi.groupby("FUNCIONALIDADE")["value_right"].count()

FUNCIONALIDADE
company           152
educational       948
entertainment    2494
food             1455
government         84
shopping         1295
sport            1452
tourism           456
Name: value_right, dtype: int64

## Limpando e Filtrando os Motivos de Parada para o Bairro Centro

In [631]:
import glob

df_stops = GeoDataFrame(columns=["name","geometry","key","value","function"])

for i, file in enumerate(glob.glob('osm/*.csv')):
    function = file[4:-4]
    print(function)
    if function[-1] == "2":
        function = function[:-1]
        
    with open(file) as category_file:
        df_stops =df_stops.append(clean_category(function,pd.read_csv(file,index_col=0).rename(columns={"geom":"geometry"})))
        
        

bus_station
83
608
bus_stop
152
1615
traffic_signals
0
1598


In [633]:
df_polygon = df_stops[df_stops.geometry.geom_type == 'Polygon']
df_point = df_stops[(df_stops.geometry.geom_type != 'Polygon') & (df_stops.value != "traffic_signals")]

#remove polygons duplicados
df_polygon_res = geopandas.sjoin(df_polygon,df_polygon, how="inner",op="intersects").reset_index()
index = df_polygon_res[df_polygon_res["index"] < df_polygon_res["index_right"]].index_right.unique()
df_polygon = df_polygon.drop(index)

#remove points duplicados
df_point_res = geopandas.sjoin(df_point,df_polygon, how="inner",op="intersects")
df_stops = df_point.drop(df_point_res.index).append(df_polygon).append(df_stops[df_stops.value == "traffic_signals"])

In [634]:
len(df_stops)

2653

### Construindo os buffers dos motivos de parada

In [635]:
from fiona.crs import from_epsg
df_stops.crs = from_epsg(3857)
df_bairros.crs = from_epsg(3857)

In [636]:
index = geopandas.sjoin(df_bairros[df_bairros.name == "Centro"],df_stops, how="inner").index_right

In [637]:
df_stops = df_stops.loc[index]

In [649]:
df_stops["buffer10m"] = buffer = df_stops.buffer(0.0001, cap_style=3)
df_stops["buffer20m"] = buffer = df_stops.buffer(0.0002, cap_style=3)
df_stops["buffer30m"] = buffer = df_stops.buffer(0.0003, cap_style=3)
df_stops["buffer50m"] = buffer = df_stops.buffer(0.0005, cap_style=3)

In [657]:
import folium

mapOSM =  folium.Map(location=[-25.4284, -49.2733],
                    zoom_start = 13)

folium.GeoJson(buffer.geometry.to_json()).add_to(mapOSM)

folium.GeoJson(df_bairros[df_bairros.name == "Centro"].geometry.to_json()).add_to(mapOSM)
mapOSM

Garantindo que somente os pontos dentro do centro serão utilizados

In [640]:
res = []
for geom in df_stops.geometry:
    res.append(df_bairros[df_bairros.name == "Centro"].geometry.contains(geom).values[0])

In [642]:
df_stops["res"] = res

In [643]:
df_stops = df_stops[df_stops.res == True]

In [654]:
df_stops = df_stops.drop("res",axis=1)

In [656]:
df_stops.to_csv("stops.csv",index=True)

In [661]:
df_stops.value.value_counts()

traffic_signals    211
bus_stop            91
bus_station         18
Name: value, dtype: int64